In [1]:
# 导入所需的库
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
import os
from sklearn.metrics import mean_squared_error

In [2]:
raw_train_data = pd.read_csv(r"C:\Users\yubin\Desktop\综合课程设计3\锂电池\train_data.csv")
raw_test_data = pd.read_csv(r"C:\Users\yubin\Desktop\综合课程设计3\锂电池\test_data.csv")
raw_dev_data = pd.read_csv(r"C:\Users\yubin\Desktop\综合课程设计3\锂电池\dev_data.csv")

In [11]:
# 删除第一列
train_data = raw_train_data.drop(raw_train_data.columns[0], axis=1)
dev_data = raw_dev_data.drop(raw_dev_data.columns[0], axis=1)
test_data = raw_test_data.drop(raw_test_data.columns[0], axis=1)
t = test_data[test_data['SoH'] != -100]
d = dev_data[dev_data['SoH'] != -100]
combined_df = pd.concat([t, d])
train_data=pd.concat([train_data,combined_df])
# 将值为0的数替换为NaN
train_data.replace(0, np.nan, inplace=True)
#训练集y
train_y=train_data['SoH']
#填补缺失值
train_data['CCCT'] = train_data['CCCT'].fillna(train_data['CCCT'].mean())
train_data['CVCT'] = train_data['CVCT'].fillna(train_data['CVCT'].mean())
train_data=train_data.drop(columns=['SoH'])
# 对"XX"列进行独热编码
train_data = pd.get_dummies(train_data, columns=["CS_Name"])

col_to_multiply = {'cycle','CS_Name_CS2_35','CS_Name_CS2_36','CS_Name_CS2_37','CS_Name_CS2_38'}
# 获取除了 col_to_multiply 列之外的所有列
cols_to_multiply_with = train_data.drop(col_to_multiply, axis=1).columns
# 循环相乘
for col in cols_to_multiply_with:
    new_col_name = f"cycle_mul_{col}"
    train_data[new_col_name] = train_data['cycle'] * train_data[col]

train_data['log_cycle'] = np.log(train_data['cycle'])

In [13]:
t_data = test_data[test_data['SoH'] == -100]
#填补缺失值
t_data['CCCT'] = t_data['CCCT'].fillna(t_data['CCCT'].mean())
t_data['CVCT'] = t_data['CVCT'].fillna(t_data['CVCT'].mean())
t_data=t_data.drop(columns=['SoH'])
t_data = pd.get_dummies(t_data, columns=["CS_Name"])

# 选择要循环相乘的列
col_to_multiply = {'cycle','CS_Name_CS2_35','CS_Name_CS2_36','CS_Name_CS2_37','CS_Name_CS2_38'}
# 获取除了 col_to_multiply 列之外的所有列
cols_to_multiply_with = t_data.drop(col_to_multiply, axis=1).columns
# 循环相乘
for col in cols_to_multiply_with:
    new_col_name = f"cycle_mul_{col}"
    t_data[new_col_name] = t_data['cycle'] * t_data[col]
t_data['log_cycle'] = np.log(t_data['cycle'])

C:\Users\yubin\AppData\Local\Temp\ipykernel_32\296861302.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t_data['CCCT'] = t_data['CCCT'].fillna(t_data['CCCT'].mean())
C:\Users\yubin\AppData\Local\Temp\ipykernel_32\296861302.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t_data['CVCT'] = t_data['CVCT'].fillna(t_data['CVCT'].mean())


In [14]:

# 建立XGB模型
model = xgb.XGBRegressor(
    n_estimators=1000,  # 迭代次数
    learning_rate=0.01,  # 学习率
    objective='reg:squarederror',  # 损失函数
    random_state=42
)


train_x=train_data;

t_data_x=t_data;

model.fit(train_x, train_y)
test_y = model.predict(t_data_x)


# rmse = np.sqrt(mean_squared_error(test_y, train_y))
# rmse

In [15]:
# 假设 test_y 是一个一维数组，你可以将其转换为 DataFrame 对象
test_y_df = pd.DataFrame({'result': test_y})
test_=raw_test_data[raw_test_data['SoH'] == -100]
test_ = test_.reset_index(drop=True)
# 按列拼接预测结果和 t_data_x
merged_df = pd.concat([test_, test_y_df],axis=1)
# merged_df.rename(columns={'Unamed: 0': 'first'}, inplace=True)
selected_columns = ['Unnamed: 0', 'cycle', 'CS_Name','result']
# 选择特定列
new_df = merged_df.loc[:, selected_columns]
new_df.to_csv('result.csv', index=False)
# # 保存拼接后的结果到新的 CSV 文件
# merged_df.to_csv('merged_result.csv', index=False)